In [22]:
import redis 

client = redis.Redis('host.docker.internal', 6379, 0)

client.flushdb()

True

In [23]:
import json
import numpy as np
import threading
import time
from concurrent.futures import ThreadPoolExecutor
from pcomp.kafka_handlers import KafkaProducerHandler, KafkaConsumerHandler, KafkaConsumerHandlerNeuron
from pcomp.activation_functions import ACTIVATIONS, relu, softmax
from pcomp.redis_utils import RedisHandler
from pcomp.parser import parse_layer_coordinator_message, parse_layer_message
from pcomp.avro_utils import avro_serialize, avro_deserialize
from pcomp.neurons_accumulator import NeuronsAccumulator

# Kafka Configuration
KAFKA_BROKER = 'kafka:29092'

In [24]:
class Neuron(threading.Thread):
    def __init__(self, layer_id, neuron_id, weights, bias, activation, is_final_layer):
        threading.Thread.__init__(self)
        self.layer_id = layer_id
        self.layer_id_num = int(self.layer_id.replace("layer_", ""))
        self.neuron_id = neuron_id
        self.weights = np.array(weights)
        self.bias = np.array(bias)
        self.activation = activation
        self.activation_func = ACTIVATIONS.get(activation, relu)
        self.is_final_layer = is_final_layer
        self.redis_handler = RedisHandler('host.docker.internal', 6379, 0, 20)
        self.producer = None
        self.neuron_acc = None
        if self.layer_id == 'layer_0':
            self.neuron_acc = -1
        elif self.layer_id == 'layer_1':
            self.neuron_acc = 128
        else:
            self.neuron_acc = 10
        

    def fetch_input(self, image_id):
        if self.neuron_acc == -1:
            key = f"streams:{image_id}:initial_data"
            return np.frombuffer(self.redis_handler.get(key), dtype=np.float64)
        else:
            key = f"streams:{image_id}:outputs:{self.layer_id_num - 1}"
            return self.redis_handler.get_output_vector(key, self.neuron_acc)

    def process_and_send(self, image_id, input_data):
        z = np.dot(input_data, self.weights) + self.bias
        output = z if self.is_final_layer else self.activation_func(z)
        self.redis_handler.hset_expiry(f"streams:{image_id}:outputs:{self.layer_id_num}", str(self.neuron_id), output, 60)
        msg = f"{image_id}"
        self.producer.send(msg)

    def run(self):
        # Instantiate Kafka consumer and producer inside the thread.
        consumer = KafkaConsumerHandler(f'layer-{self.layer_id_num}', KAFKA_BROKER, group_id=f"{self.neuron_id}_{self.layer_id_num}_group")
        self.producer = KafkaProducerHandler(KAFKA_BROKER, f'layer-{self.layer_id_num}-streams')
        last_msg_time = time.time()
        while True:
            got_message = False
            for message in consumer.consume():
                got_message = True
                last_msg_time = time.time()
                image_id_str = message
                image_id = int(image_id_str)
                try:
                    input_data = self.fetch_input(image_id)
                    self.process_and_send(image_id, input_data)
                except Exception:
                    pass
            if not got_message and (time.time() - last_msg_time > 10):
                consumer.commit()
                consumer.close()
                self.producer.close()
                self.redis_handler.close()
                break


class NeuronOutput(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
        self.redis_handler = RedisHandler('host.docker.internal', 6379, 0, 2)
        self.last_layer_id_num = 1
        self.neuron_acc = 10

    def run(self):
        consumer = KafkaConsumerHandler(f'layer-output', KAFKA_BROKER, group_id=f"neuron_output_coord_group")
        last_msg_time = time.time()
        while True:
            got_message = False
            for message in consumer.consume():
                got_message = True
                last_msg_time = time.time()
                image_id = int(message)
                try:
                    key = f"streams:{image_id}:outputs:{self.last_layer_id_num}"
                    outputs = self.redis_handler.get_output_vector(key, self.neuron_acc)
                    prediction = int(np.argmax(outputs))
                    self.redis_handler.hset('streams:predictions', image_id, prediction)
                except Exception:
                    pass
            if not got_message and (time.time() - last_msg_time > 15):
                consumer.commit()
                consumer.close()
                self.redis_handler.close()
                break

# Load network and dataset
data = json.load(open("node_based_model.json"))
#df = pd.read_csv('data/mnist.csv').head(10)

neurons = []

for layer_name, layer_info in data.items():
    neurons += [Neuron(layer_id=layer_name, neuron_id=i, weights=node['weights'], bias=node['biases'], activation=node['activation'], is_final_layer=(layer_name == list(data.keys())[-1])) for i, node in enumerate(layer_info['nodes'])]

neuron_output = NeuronOutput()

# Start all threads
for thread in neurons:
    thread.start()

neuron_output.start()

print("Threads started")

Threads started


%4|1744241922.106|OFFSET|rdkafka#consumer-1662| [thrd:main]: layer-output [0]: offset reset (at offset 68 (leader epoch 0), broker 1) to offset END (leader epoch -1): fetch failed due to requested offset not available on the broker: Broker: Offset out of range
%4|1744241923.735|OFFSET|rdkafka#consumer-1642| [thrd:main]: layer-1 [6]: offset reset (at offset 2 (leader epoch 0), broker 1) to offset END (leader epoch -1): fetch failed due to requested offset not available on the broker: Broker: Offset out of range
%4|1744241923.737|OFFSET|rdkafka#consumer-1643| [thrd:main]: layer-1 [6]: offset reset (at offset 2 (leader epoch 0), broker 1) to offset END (leader epoch -1): fetch failed due to requested offset not available on the broker: Broker: Offset out of range
%4|1744241923.739|OFFSET|rdkafka#consumer-1647| [thrd:main]: layer-1 [6]: offset reset (at offset 2 (leader epoch 0), broker 1) to offset END (leader epoch -1): fetch failed due to requested offset not available on the broker: Br

In [21]:
# Wait for all threads to complete
for thread in neurons:
    thread.join()

neuron_output.join()

print("Threads finished")

Threads finished


In [25]:
import redis

# Connect to Redis
r = RedisHandler('host.docker.internal', 6379, 0)

# Get hashes from Redis
images_label = r.hgetall('streams:images_label')
predictions = r.hgetall('streams:predictions')

# Decode bytes to string
images_label = {k.decode(): v.decode() for k, v in images_label.items()}
predictions = {k.decode(): v.decode() for k, v in predictions.items()}

# Calculate accuracy
correct = 0
total = len(images_label)

for field, label_val in images_label.items():
    pred_val = predictions.get(field, None)
    if pred_val == label_val:
        correct += 1

accuracy = (correct / total) * 100 if total > 0 else 0

print(f'Accuracy: {accuracy:.2f}% ({correct}/{total})')


Accuracy: 99.00% (99/100)
